### Fonction qui permet de retourner la variable sur laquelle on branche

In [1]:
function calculate_branching(x)
    donneur = 0
    receveur = 0
    found = false
    for i in 1:nv(new_inst.graph)
        for j in 1:nv(new_inst.graph)
            if (mod(x[i][j],1) > ϵ)
                donneur = i
                receveur = j
                found=true
                break
            end
        end
        found && break
    end
    return (donneur,receveur)
end

calculate_branching (generic function with 1 method)

### Fonction qui résout le Branch and Price

In [2]:
function solve_BP()
    global column_pool=Array{Array{Array{Int,1},1},1}(undef,nv(new_inst.graph))
    # C'est un vecteur à deux dimensions
    # column_pool contient tous les pattern
    # column_pool[i] est un pattern
    # Les patterns sont tels que :
    #   - le premier élément est la contribution à la valeur objective
    #   - les éléments 2 à n+1 sont les éléments 0 et 1 qui dépend si la paire patient-donneur est dans le cycle
    #   - le dernier élément contribue à la contrainte de convexité, il y a seulement une paire impliquée dans tous les cycles choisi
    for l in 1:nv(new_inst.graph)
        column_pool[l]=Array{Array{Int,1},1}()
        push!(column_pool[l],vcat(-10000,ones(Int,nv(new_inst.graph)))) # motif artificiel
        push!(column_pool[l],vcat(0,zeros(Int,nv(new_inst.graph)))) # motif vide
    end
    # Initialisation des bornes inférieure et supérieure
    global UB = Inf  # ou la valeur d'une solution heuristique 
    global LB = -Inf # ou la valeur de la relaxation initiale
    
    global Queue = Vector{Int}() # les noeuds que l'on doit traiter, vecteur d'entiers
    global tree = Vector{TreeNode}() # l'arbre de Branch and Price, vecteur de TreeNode
    
    # Initialisation du Branch and Brice
    push!(tree,TreeNode(0,[],-Inf,[],[]))   
    push!(Queue,1) # indices des noeuds à traiter

    # Algorithme de Branch and Price
    while length(Queue)>0
        println("Liste actuelle des noeuds à traiter:", Queue)
        
        # Traitement du dernier noeud
        current = Queue[end]
        
        # Résolution du problème maître restreint
        x = Process_Node(current) # retourne la solution optimale ou [] si le problème est infaisable
        
        # Si le noeud est faisable et que la borne inférieure est intéressante, on branche
        if size(x,1)!=0 && tree[current].lb <= UB
            # On cherche la variable de branchement
            (donneur,receveur) = calculate_branching(x) # retourne la paire tel que z_c est fractionnaire ou 0
            
            # Ajout des noeuds enfants si on a une variable de branchement
            if (donneur,receveur)!=(0,0)
                println("Deux noeuds sont créés en branchant sur la variable x[$donneur,$receveur]")
                # Noeud 1
                push!(tree,TreeNode(current,[],tree[current].lb,vcat((donneur,receveur),tree[current].setzero),tree[current].setone))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
                # Noeud 2
                push!(tree,TreeNode(current,[],tree[current].lb,tree[current].setzero,vcat((donneur,receveur),tree[current].setone)))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
            else
                println("La solution optimale de la relaxation était entière")
                println("Solution faisable avec valeur $(tree[current].lb) trouvée")
            end
        else
            println("Le noeud n'est pas faisable ou il est élagué")
        end
            
        # Caclul de la borne inférieure globale
        global LB = tree[current].lb
        for i in Queue
            if tree[i].lb <= LB
                global LB=tree[i].lb
            end
        end
        
        # Affichage des bornes supérieure et inférieure actuelle
        println("LB=$LB,UB=$UB")
        
        # Suppression du noeud qui a été traité
        deleteat!(Queue,findfirst(x -> x == current, Queue))
        
        # Suppression des noeuds avec une borne non intéressante
        deleteat!(Queue,unique(nodestobedeleted))
        
        # Arrêt de l'algorithme si l'optimalité est presque atteinte
        if (2*(UB-LB)/(UB+LB))<=ϵ
            break
        end
    end
end

solve_BP (generic function with 1 method)